In [ ]:
# Main Imports
from ipynb.fs.full.GastricFunctions import *
#from GastricFunctions_py import *
import numpy as np
import pandas as pd
# Util function for convert-to (0 - 1) range
def rescale(arr):
    arr_min = arr.min()
    arr_max = arr.max()
    return (arr - arr_min) / (arr_max - arr_min)

def rescale_AB(arr):
    arr_min = arr.min()
    arr_max = arr.max()
    return ( (arr - arr_min) / (arr_max - arr_min)  ) * 255 - 128

In [ ]:
# Loading Results dataframe
# Prepare this dataframe (total results) only once
# total_results_dataframe = pd.DataFrame(columns=["Patient to be predicted/left out" , "Poor Survival Subpopulation" , "Moderate Survival Subpopulation" , "Good Survival Subpopulation", "Predicted Survivability", "Surv(months)" , "Survival Status" , "Number of Clusters"  , "SAM Features for each tSNE run on new subset"])
#total_results_dataframe_SVM = pd.DataFrame(columns=["Patient to be predicted/left out" , "Poor Survival Subpopulation" , "Moderate Survival Subpopulation" , "Good Survival Subpopulation", "Predicted Survivability", "Surv(months)" , "Survival Status" , "Number of Clusters"  , "SAM Features for each tSNE run on new subset"])
#total_results_dataframe_KNN = pd.DataFrame(columns=["Patient to be predicted/left out" , "Poor Survival Subpopulation" , "Moderate Survival Subpopulation" , "Good Survival Subpopulation", "Predicted Survivability", "Surv(months)" , "Survival Status" , "Number of Clusters"  , "SAM Features for each tSNE run on new subset"])

# Only load if needed to load total results from previous runs to continue on
total_results_dataframe_SVM = pd.read_csv("total_GastricCancer_results_SVM.csv")
total_results_dataframe_KNN = pd.read_csv("total_GastricCancer_results_KNN.csv")


In [ ]:
# Reading Data
HE_image , MSI_data_cube , goodlist, peak_list, pixel_to_sample_ID, sample_only_data, sample_ID_pixels = ReadGastricData()

In [ ]:
## Selecting Patient as test
testPatient_ID = 63
#-------------------------------------------------------
# Splitting data into train and test and scaled and non_scaled
train_scaled_data,train_data,train_ID_pixels,test_scaled_data,test_data,test_ID_pixels = SplitData(ID = testPatient_ID , sample_only_data = sample_only_data, sample_ID_pixels = sample_ID_pixels )

In [ ]:
# Run tSNE Model

# tsne_results = tSNE(train_scaled_data)
# np.save("patient_TSNE/GastricCancer_tSNE_Model_Patient_{}".format(testPatient_ID) , tsne_results)

In [ ]:
# Load tSNE model instead of running model if needed
tsne_results = np.load("patient_TSNE/GastricCancer_tSNE_Normalized_Model_Patient_{}.npy".format(testPatient_ID))

In [ ]:
# # Visualize tSNE

# tsne_df = pd.DataFrame(data=tsne_results[:],columns=["tSNE_dim1","tSNE_dim2","tSNE_dim3"])
# tsne_df['pixel_ID'] = train_ID_pixels
# tsne_df['L_Component'] = tsne_results[:,0]
# tsne_df['A_Component'] = tsne_results[:,1]
# tsne_df['B_Component'] = tsne_results[:,2]

# # Converting tSNE dimensions to be represented 

# LAB_mapping = [ tsne_df['L_Component'] , tsne_df['A_Component'] , tsne_df['B_Component'] ]
# LAB_colors = np.asarray(LAB_mapping)
# LAB_colors = LAB_colors.swapaxes(0,1)

# L = rescale(LAB_colors[:,0]) * 100
# A = rescale_AB(LAB_colors[:,1])
# B = rescale_AB(LAB_colors[:,2])
# LAB_colors = np.asarray( [ L , A , B ] )
# LAB_colors = LAB_colors.swapaxes(0,1)
# RGB_colors = lab2rgb(LAB_colors)

# ax = plt.figure(figsize=(16,10)).gca(projection='3d')
# scatter_3d = ax.scatter(
#     xs=tsne_df['tSNE_dim1'], 
#     ys=tsne_df['tSNE_dim2'], 
#     zs=tsne_df['tSNE_dim3'], 
#     c = RGB_colors,
#     s = 5,
#     alpha = 1,
# )

# ax.set_xlabel('tSNE_dim1')
# ax.set_ylabel('tSNE_dim2')
# ax.set_zlabel('tSNE_dim3')
# ax.set_title("tSNE colored map")

# # fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

# # elev is z-axis, +ve moves up, -ve moves down
# # azim is x axis, +ve moves right, -ve moves left
# ax.view_init(elev = 20 , azim = -160) 
# plt.tight_layout()

In [ ]:
# Applying KMeans from 3 to 8 clusters
kmeans_3 , kmeans_4, kmeans_5, kmeans_6, kmeans_7, kmeans_8 = KMeans_results(tsne_results)

In [ ]:
# Reading Clinical data to be applied in survival analysis
Clinical_data = ReadClinicalData(testPatient_ID , "ClinicalData.xlsx")
Clinical_data_full = pd.read_excel("ClinicalData.xlsx")

Clinical_data["Surv_time"] = round(Clinical_data["Surv_time"] / 30 , 2)
Clinical_data_full["Surv_time"] = round(Clinical_data_full["Surv_time"] / 30 , 2)

In [ ]:
# Create dict to save kmeans results and select them later
kmeans_dict = { '3_Clusters' : kmeans_3.labels_ , '4_Clusters': kmeans_4.labels_ , '5_Clusters': kmeans_5.labels_ , '6_Clusters': kmeans_6.labels_ , '7_Clusters': kmeans_7.labels_ , '8_Clusters': kmeans_8.labels_ }
colors = ["red" , "green" , "yellow" , "blue", "violet" , "Indigo" , "Orange" , "grey" , "black" , "pink"]

In [ ]:
# # Perform Kaplan Meier, Select the correct kmeans that shows best results from survival analysis, must have two completely seperetable survival lines
# for i in range(3,9):
#     labels = kmeans_dict[str(i)+'_Clusters']
#     Clinical_data_new = KaplanMeierSurvivalFitter(labels , colors = colors , sample_ID_pixels = train_ID_pixels , Clinical_data = Clinical_data)

#     # un-comment if interested in hiding all kaplan meier plots
#     #plt.close()

In [ ]:
# Print all pvalues for all clusters
for i in range(3,9):
    selected_state = kmeans_dict[str(i) + "_Clusters"]
    labels = kmeans_dict[str(i) + "_Clusters"]
    Clinical_data_new = KaplanMeierSurvivalFitter(labels , colors = colors, sample_ID_pixels = train_ID_pixels , Clinical_data = Clinical_data)
    plt.close()
    Results = LogRankTest_PrintValues(labels , Clinical_data_new, printResults=False)
    pvalue , SignCluster = SignificantClusters(labels, Results)
    print("at KMeans = {} , pvalue is: {}".format(i,pvalue))

In [ ]:
## Select the correct kmeans that shows best results from survival analysis, must have two completely seperetable survival lines
# 3 or 4 or 5 or 6 or 7 or 8 
selected_clusters_value = 6
# ----------------------------------------------------------------------------------

In [ ]:
# Using Selected KMeans, apply LogRank test and Kaplan Meier to be used in Cox Hazard

selected_state = kmeans_dict[str(selected_clusters_value) + "_Clusters"]
labels = kmeans_dict[str(selected_clusters_value) + "_Clusters"]

Clinical_data_new = KaplanMeierSurvivalFitter(labels , colors = colors, sample_ID_pixels = train_ID_pixels , Clinical_data = Clinical_data)
plt.close()

Results = LogRankTest_PrintValues(labels , Clinical_data_new, printResults=False)

pvalue , SignCluster = SignificantClusters(labels, Results)


print("Significant Clusters are: Cluster " + str(SignCluster[0]+1) + " and Cluster " + str(SignCluster[1]+1))
print(pvalue)

In [ ]:
# Plot p-value with Kaplan Meier graphs of selected KMeans
print("Significant Clusters are: " , np.add(SignCluster,1))

# Possible to manually input the Significant needed to be plotted
#SignCluster = [1 , 2 , 4 , 6 ]

Clinical_data_new = KaplanMeierSurvivalFitter(labels , colors = colors, sample_ID_pixels = train_ID_pixels, Clinical_data = Clinical_data)
plt.text(0.30,0.05, "{}{}".format("P-value= ", pvalue) , fontsize = 12,fontweight="bold",bbox = dict(facecolor = 'lightblue', alpha = 0.5))


Clinical_data_new= KaplanMeierSurvivalFitter(labels , colors = colors, sample_ID_pixels = train_ID_pixels, Clinical_data = Clinical_data,SignCluster=SignCluster)
plt.text(0.30,0.05, "{}{}".format("P-value= ", pvalue) , fontsize = 12,fontweight="bold",bbox = dict(facecolor = 'lightblue', alpha = 0.5))

In [ ]:
# Clusters_to_be_drawn = [ 5 , 4 ]
# Clinical_data_new= KaplanMeierSurvivalFitter(labels , colors = colors, sample_ID_pixels = train_ID_pixels, Clinical_data = Clinical_data,SignCluster=Clusters_to_be_drawn)
# plt.text(0.30,0.05, "{}{}".format("P-value= ", round(Results[Clusters_to_be_drawn[0]][Clusters_to_be_drawn[1]].p_value,2)) , fontsize = 12,fontweight="bold",bbox = dict(facecolor = 'lightblue', alpha = 0.5))

In [ ]:
# Apply CoxHazard Fitter
cph = CoxHazardFitter(labels , Clinical_data = Clinical_data_new)

labels_count = len(np.unique(labels))

Clusters = [ [] for _ in range(labels_count)]
for cluster_label in range(labels_count):
    Clusters[cluster_label] = Clinical_data_new.query("Belong_Cluster_" + str(cluster_label+1) + " == 1")

# Clears output, can be commented if needed
from IPython.display import clear_output
clear_output()
plt.close()

In [ ]:
# Plot CoxHazard bar graph to show hazardous value of each cluster
labels_count = len(np.unique(labels))

Clusters = [ [] for _ in range(labels_count)]
for cluster_label in range(labels_count):
    Clusters[cluster_label] = Clinical_data_new.query("Belong_Cluster_" + str(cluster_label+1) + " == 1")
    
max_tick_value = max(cph.hazard_ratios_)
COLORS = ["yellow" , "red"]
CoxHazardBarPlot(cph = cph, colors = COLORS , max_tick_value = max_tick_value, clusters = Clusters)

In [ ]:
# Check for signifiance between 2 clusters using cluster_value - 1 if needed
Results[4][5]

In [ ]:
# Using coxhazard to determine the significant clusters, check coxhazard before relying on these values
hazardous_cluster_label = np.argmax(cph.hazard_ratios_) # Find the max hazard ratio's index and that will be the hazardous cluster label
survival_cluster_label = np.argmin(cph.hazard_ratios_) # Find the min hazard ratio's index and that will be the highest survival cluster label

print("Hazardous cluster label's index is {} and it is cluster {}".format(hazardous_cluster_label,hazardous_cluster_label+1))
print("High Survival cluster label's index is {} and it is cluster {}".format(survival_cluster_label,survival_cluster_label+1))

In [ ]:
# Manual input cluster labels if needed
survival_cluster_label = 1
hazardous_cluster_label = 0

print("Hazardous cluster label's index is {} and it is cluster {}".format(hazardous_cluster_label,hazardous_cluster_label+1))
print("Survival cluster label's index is {} and it is cluster {}".format(survival_cluster_label,survival_cluster_label+1))

In [ ]:
# Perform SAM Analysis, Run R file after this cell
protein_dataframe = SAM_Analysis(labels, Clinical_data, train_ID_pixels, hazardous_cluster_label, train_data, peak_list)
protein_dataframe.to_csv("GastricCancerAverageSpectrumValues.csv",index=False)

# Run R file in order to generate significant proteins using the csv file

In [ ]:
#Need to read significant protein file outputted from R, Create Target labels to be used for KNN and SVM Models

SAM_protein = readSignificantProteins("significant_gastric_proteins.json",delete=False)
SAM_protein = SAM_protein + readSignificantProteins("significant_gastric_proteins_up.json",delete=False)

SAM_protein_indices=[]
for i in range(len(peak_list)):

    if peak_list[i][0] in SAM_protein:
        SAM_protein_indices.append(i)

train_scaled_selected_proteins_data = train_scaled_data[:,SAM_protein_indices]
test_scaled_selected_proteins_data = test_scaled_data[:,SAM_protein_indices]

train_selected_proteins_data = train_data[:,SAM_protein_indices]
test_selected_proteins_data = test_data[:,SAM_protein_indices]

train_labels = TargetLabelsCreation(labels , Clinical_data, train_ID_pixels, hazardous_cluster_label , survival_cluster_label)

print(SAM_protein)

In [ ]:
# SVM Model , predicting labels
# Test rbf at c = 0.1 , 1 , 10
# Test linear at c = 0.1 , 1 , 10
y_pred_SVM = SVM("rbf" , 1 , train_selected_proteins_data , train_labels , test_selected_proteins_data)

In [ ]:
# Calculating SVM Probability survival values
Poor_Surv, Moderate_Surv , High_Surv = ProbabilityCalc(y_pred_SVM)
Surv_SVM = [Poor_Surv , Moderate_Surv, High_Surv]

In [ ]:
# KNN Model, predicting labels
y_pred_KNN = KNN(3 , train_selected_proteins_data, train_labels, test_selected_proteins_data)

In [ ]:
# Calculating KNN Probability values
Poor_Surv, Moderate_Surv , High_Surv  = ProbabilityCalc(y_pred_KNN)
Surv_KNN = [Poor_Surv , Moderate_Surv, High_Surv]

In [ ]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(train_selected_proteins_data, train_labels)
y_pred = clf.predict(test_selected_proteins_data)

Poor_Surv, Moderate_Surv , High_Surv  = ProbabilityCalc(y_pred)

In [ ]:
# Selecting KNN as outputted results
Selected_model = Surv_KNN
Poor_Surv = round(Selected_model[0],2)
Moderate_Surv = round(Selected_model[1],2)
High_Surv = round(Selected_model[2],2)

In [ ]:
# Run after model complete to save the result in the total results dataframe, outputting KNN results in dataframe
total_results_dataframe_KNN = OutputDataframe(total_results_dataframe_KNN, testPatient_ID , Clinical_data_full, Poor_Surv , Moderate_Surv, High_Surv ,selected_clusters_value, SAM_protein)

In [ ]:
# Preview KNN results
total_results_dataframe_KNN

In [ ]:
# Saving KNN results in CSV file
# Only run if needed to save and load at a later time
total_results_dataframe_KNN.to_csv("total_GastricCancer_results_KNN.csv" , index = False)

In [ ]:
# Selecting SVM as outputted results
# Selected Gastric probabilities, select KNN or SVM results
Selected_model = Surv_SVM
Poor_Surv = round(Selected_model[0],2)
Moderate_Surv = round(Selected_model[1],2)
High_Surv = round(Selected_model[2],2)

In [ ]:
# Run after model complete to save the result in the total results dataframe, outputting SVM results in dataframe
total_results_dataframe_SVM = OutputDataframe(total_results_dataframe_SVM, testPatient_ID , Clinical_data_full, Poor_Surv , Moderate_Surv, High_Surv ,selected_clusters_value, SAM_protein)

In [ ]:
# Preview SVM results
total_results_dataframe_SVM

In [ ]:
# Saving SVM results in CSV file
# Only run if needed to save and load at a later time
total_results_dataframe_SVM.to_csv("total_GastricCancer_results_SVM.csv" , index = False)